In [1]:
import json
import sqlite3
from nltk import word_tokenize

In [2]:
from more_itertools import split_after, split_at

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\demid\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
CLAUSE_KEYWORDS = ('select', 'from', 'where', 'group', 'order', 'limit', 'intersect', 'union', 'except')
JOIN_KEYWORDS = ('join', 'on', 'as')

# WHERE_OPS = ('not', 'between', '=', '>', '<', '>=', '<=', '!=', 'in', 'like', 'is', 'exists')

# delete "in" 
WHERE_OPS = ('not', 'between', '=', '>', '<', '>=', '<=', '!=', 'like', 'is', 'exists')

UNIT_OPS = ('none', '-', '+', "*", '/')
AGG_OPS = ('none', 'max', 'min', 'count', 'sum', 'avg')
TABLE_TYPE = {
    'sql': "sql",
    'table_unit': "table_unit",
}

COND_OPS = ('and', 'or')
SQL_OPS = ('intersect', 'union', 'except')
ORDER_OPS = ('desc', 'asc')

In [5]:
cmp_op_idx2op_symbol = {0: '=', 1: '>', 2: '<'}
ctr_symbol2cmp_op_idx = {v: k for k, v in cmp_op_idx2op_symbol.items()}

agg_idx2op_name = {1: 'max', 2: 'min', 3: 'count', 4: 'sum', 5: 'avg'}
ctr_name2agg_idx = {v: k for k, v in agg_idx2op_name.items()}

In [6]:
# подавать список токенов условий 

def parse_condition(conditions):
    for tok in conditions:        
        if tok in WHERE_OPS:
            agg_tok_idx = conditions.index(tok)
            agg_tok = tok
            cond_col = conditions[:agg_tok_idx][0]
            rest_cond = ' '.join(conditions[conditions.index(agg_tok)+1:])
    return [cond_col, ctr_symbol2cmp_op_idx[agg_tok], rest_cond]
# agg_tok, agg_tok_idx, cond_col

In [7]:
from process_sql import tokenize

In [8]:
my_query = 'SELECT COUNT column_name FROM table WHERE random_col = New series began in June 2011 AND col_col > Vse HOROsho'

In [9]:
toks = tokenize(my_query)
toks

['select',
 'count',
 'column_name',
 'from',
 'table',
 'where',
 'random_col',
 '=',
 'new',
 'series',
 'began',
 'in',
 'june',
 '2011',
 'and',
 'col_col',
 '>',
 'vse',
 'horosho']

1. Нашли аггрегатор и колонку для select

In [10]:
col_to_select_with_agg = toks[1:toks.index('from')]
col_to_select_agg_op, col_to_select = col_to_select_with_agg[0], col_to_select_with_agg[1]

2. Нашли блок с условиями

In [11]:
conditions = toks[toks.index('where')+1:]
conditions

['random_col',
 '=',
 'new',
 'series',
 'began',
 'in',
 'june',
 '2011',
 'and',
 'col_col',
 '>',
 'vse',
 'horosho']

3. Разделели условия

In [12]:
с_1 = list(split_at(conditions, lambda x: x in COND_OPS))[0]
с_2 = list(split_at(conditions, lambda x: x in COND_OPS))[1]

4. Распарсили условия

In [13]:
cond_list = []
for c in list(split_at(conditions, lambda x: x in COND_OPS)):
    cond_list.append(parse_condition(c))
    
cond_list

[['random_col', 0, 'new series began in june 2011'],
 ['col_col', 1, 'vse horosho']]

5. Сделали итоговый словарь 

Нужны: колонка для select, ее аггрегационная операция, условия

In [14]:
final_dict = dict()

final_dict['sel'] = col_to_select
final_dict['agg'] = ctr_name2agg_idx[col_to_select_agg_op]
final_dict['conds'] = cond_list

In [15]:
final_dict

{'sel': 'column_name',
 'agg': 3,
 'conds': [['random_col', 0, 'new series began in june 2011'],
  ['col_col', 1, 'vse horosho']]}

1. sel: название колонки, потом сделать маппинг в int

2. agg: аггрегация колонки для select, потом использовать маппинг

3. conds: [id колоники, маппинг операции, условие (строка)]